In [1]:
import pandas as pd
import pptx
from pptx.chart.data import ChartData

# Functions

### Get Map

In [2]:
def get_map():
    template_mapping_path = 'template_mapping.xlsx'
    template_mapping = pd.read_excel(template_mapping_path, index_col=[0, 1, 2])
    return template_mapping

### Get DFs

In [3]:
def get_dfs():
    dfs = {}

    paths = ['Sick over time', 'Sla over time by HMO']
    for path in paths:
        dfs[path] = pd.read_excel(path+'.xlsx', parse_dates=True, index_col=[0])

    print(dfs[paths[0]].dtypes)
    dfs[paths[0]].head()
    return dfs

### get charts as dictionary

In [4]:
def get_children_charts_dict(shapes_parent):
    charts_by_title = {}
    for shape in shapes_parent.shapes:
        try:
            children_charts_by_title = get_children_charts_dict(shape)
            charts_by_title.update(children_charts_by_title)
        except:
            pass

        if not shape.has_chart:
            continue
        
        title = shape.chart.chart_title.text_frame.text if shape.chart.has_title and shape.chart.chart_title.has_text_frame else ''
        if title == '':
            title = shape.chart.series[0].name

        charts_by_title[title] = shape.chart
    return charts_by_title


charts_by_title = {}

### modify charts content

In [5]:
def insert_data(shapes_by_text, charts_by_title, mapping_row, dfs):
    
    if (mapping_row['TYPE'] == 'chart') and (key_to_replace in charts_by_title.keys()):
        
        # duplicate existing chart
        chart_data = ChartData()
        
        # copy categories from new df's index
        chart_data.categories = dfs[script_name].index # charts_by_title[key_to_replace].plots[0].categories
        
        # copy data from new df's values
        for column in dfs[script_name].columns:
            chart_data.add_series(column, dfs[script_name][column].values)        
        
        # replace original charts with the modified duplicates
        charts_by_title[key_to_replace].replace_data(chart_data)
        

# MAIN function

In [18]:
def main():
    # get mapping matrix
    template_mapping = get_map()
    
    # get dataframes with which you intend to replace current content
    dfs = get_dfs()
    
    # get existing PP Presentation
    prezi_path = 'prezi.pptx'
    presentation = pptx.Presentation(prezi_path)

    # get existing charts from prez
    charts_by_title = {}
    for slide in presentation.slides:
        slide_charts_by_title = get_children_charts_dict(slide)
        charts_by_title.update(slide_charts_by_title)

    # insert new data to charts
#     template_mapping.apply(lambda x: insert_data(None,charts_by_title, x, dfs), axis=1)
    for i, x in template_mapping.iterrows():
        try:
            insert_data(None,charts_by_title, x, dfs)
        except Exception as e:
            raise e
    
    # Save and Quit
    presentation.save(prezi_path)
    print('Done')
   

## Apply & Save

In [19]:
if __name__ == "__main__":
    main()


SICK    float64
dtype: object
Done
